## Early merge with soft parameters ##

In [1]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import  Dataset
from PIL import Image
from torchvision import transforms
import random
from sklearn.metrics import accuracy_score

In [2]:
class early_dataloader(Dataset):
    def __init__(self, data_dir , train):
        self.data_dir = data_dir
        self.all_classes = ['NotSnow','Snow']
        self.class_indexes = {cls: idx for idx, cls in enumerate(self.all_classes)}
        self.dataset = self.dataset_maker(train)
        self.length = len(self.dataset)
    def dataset_maker(self, train): # this will compile all the folder paths and the corresponding classes in a tuple to be stored in the class
        data_list = []
        for class_name in self.all_classes:
            all_class_folders =os.path.join(self.data_dir, class_name)
            for fold in os.listdir(all_class_folders):
                if fold == ".DS_Store":
                    continue
                sample_path_n_class = os.path.join(all_class_folders, fold)
                sample = (sample_path_n_class, self.class_indexes[class_name])
                data_list.append(sample)
        random.shuffle(data_list)
        if train:
            return data_list[:int(0.8*len(data_list))]
        else:
            return data_list[int(0.8*len(data_list)):]
    def __len__(self):
        return self.length
    
    def __getitem__(self, index):
        sample_path, sample_class = self.dataset[index]
       
        pic1 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '0.png')).convert('L')) # image 1 in tensor
        pic2 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '1.png')).convert('L')) # image 2 in tensor
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
        ])
        pic1 = transform(pic1)
        pic2 = transform(pic2)
        
        return pic1.unsqueeze(0) , pic2.unsqueeze(0), sample_class

In [3]:
e_soft_training = early_dataloader("./dataset/", True)
e_soft_testing = early_dataloader("./dataset/", False)

In [4]:
sample4 = e_soft_training[242]
print(sample4[0].shape)
print(sample4[1].shape)
print(sample4[2])

torch.Size([1, 1, 224, 224])
torch.Size([1, 1, 224, 224])
0


In [5]:
class early_soft_NN(nn.Module):
    def __init__(self):
        super(early_soft_NN, self).__init__()
        self.conv1 = nn. Conv2d(1, 2, kernel_size=2, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn. Conv2d(4, 4, kernel_size=2, stride=1)
        self.pool3 = nn.MaxPool2d(kernel_size=4, stride=1)
        self.conv4 = nn. Conv2d(4, 2, kernel_size=3, stride=1)
        self.pool4 = nn.MaxPool2d(kernel_size=5, stride=4)
        self.fc1 = nn.Linear (1352, 2)
        
    def forward (self, x1,x2):
        x1 = F.relu(self.conv1(x1))
        x1 = self.pool(x1)
        x2 = F.relu(self.conv1(x2))
        x2 = self.pool(x2)
        x= torch.cat((x1, x2), dim=1)
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        x = F.relu(self.conv4 (x))
        x = self.pool4(x)
        x=torch.flatten (x, 1)
        x = self.fc1(x)
        return x

In [6]:
# hyper parameters
learning_rate_5= 0.001
epochs_5 = 8

In [7]:
# training the neural net with the stacking dataset
model5 = early_soft_NN()
criterion5 = nn.CrossEntropyLoss()
optimiser5 = optim.Adam(model5.parameters(), lr = learning_rate_5)


In [8]:
# model params
total_params = sum(p.numel() for p in model5.parameters())
print(f'the total number of parameters in this model are {total_params}')

the total number of parameters in this model are 2858


In [9]:
model5.train()
for epoch in range(epochs_5):
    running_loss = 0 
    predicted_list = []
    labels_list = []
    print(f'this is epoch number {epoch}')
    for i, (input1,input2, label) in enumerate(e_soft_training):
        optimiser5.zero_grad()
        labels_list.append(label)
        label = torch.tensor(label).view(-1)
        if (i % 1000 == 0):
            print(f'this is the {i} iteration')
        outputs = model5(input1, input2)
        loss = criterion5(outputs, label)
        loss.backward()
        optimiser5.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        predicted_list.append(predicted.item())
        if i % 1000 == 999:   
            print('loss: %.3f accuracy: %.3f' %(running_loss/10, 100 * accuracy_score(labels_list, predicted_list)))
            running_loss = 0

this is epoch number 0
this is the 0 iteration
loss: 63.336 accuracy: 72.100
this is the 1000 iteration
loss: 54.946 accuracy: 72.900
this is the 2000 iteration
this is epoch number 1
this is the 0 iteration
loss: 23.994 accuracy: 91.000
this is the 1000 iteration
loss: 21.003 accuracy: 91.800
this is the 2000 iteration
this is epoch number 2
this is the 0 iteration
loss: 19.811 accuracy: 92.700
this is the 1000 iteration
loss: 17.320 accuracy: 93.400
this is the 2000 iteration
this is epoch number 3
this is the 0 iteration
loss: 16.821 accuracy: 94.100
this is the 1000 iteration
loss: 23.926 accuracy: 93.800
this is the 2000 iteration
this is epoch number 4
this is the 0 iteration
loss: 15.354 accuracy: 94.800
this is the 1000 iteration
loss: 14.130 accuracy: 94.900
this is the 2000 iteration
this is epoch number 5
this is the 0 iteration
loss: 14.101 accuracy: 95.500
this is the 1000 iteration
loss: 12.888 accuracy: 95.700
this is the 2000 iteration
this is epoch number 6
this is the

In [10]:
model5.eval()
loss = 0 
labels_list = []
predicted_list = []
with torch.no_grad():
    for i, (input1,input2, label) in enumerate(e_soft_testing):
        label = torch.tensor(label).view(-1)
        labels_list.append(label.item())
        outputs = model5(input1, input2)
        loss += criterion5(outputs, label).item()
        predicted = outputs.argmax(dim = 1 , keepdim = True)
        predicted_list.append(predicted.item())

In [11]:
print(f'the model accuracy is {accuracy_score(labels_list, predicted_list)}')

the model accuracy is 0.969187675070028
